In [ ]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage
from langchain_community.chat_models import ChatOpenAI
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


chat = ChatOpenAI(model='LLaMA_CPP',
                    temperature=0,
                    openai_api_key="sk-no-key-required", 
                    base_url='http://localhost:8080/v1',
                    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))  






In [ ]:
%pip install langchain-openai

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="LLaMA_CPP",
                    openai_api_key="sk-no-key-required", 
                    base_url='http://localhost:8080/v1')
output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(chain)



chain.invoke({"topic": "ice cream"})